Toolcalling enables LLMs to interact with external enviroments through a dynamic interface. Tool calling helps to choose an appropriate tool and make necessary arguments to executing the tool.

In tool calling we can equip LLM to select a tool among a set of tools that we have prepared for it and make them to choose proper input for the tool and inject the inputs and return the reults to us. We can also do the tool calling for in the RAG process.

In [1]:
!pip install openai
!pip install llama-index


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [3]:
import nest_asyncio
nest_asyncio.apply()

When using an agent or LLM with function calling, the tool selected (and the arguments written for that tool) rely strongly on the tool name and description of the tools purpose and arguments. Spending time tuning these parameters can result in larges changes in how the LLM calls these tools.

A Tool implements a very generic interface - simply define __call__ and also return some basic metadata (name, description, function schema).

We offer a few different types of Tools:

FunctionTool: A function tool allows users to easily convert any user-defined function into a Tool. It can also auto-infer the function schema.

QueryEngineTool: A tool that wraps an existing query engine. Note: since our agent abstractions inherit from BaseQueryEngine, these tools can also wrap other agents.

Community contributed ToolSpecs that define one or more tools around a single service (like Gmail)

Utiltiy tools for wrapping other tools to handle returning large amounts of data from a tool

On the other words, these tools make other tools properly callable in the LLM. In the following example we have defined two tools in python then made it callable for the LLM with the FunctionTool.

In [4]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

C:\Users\moham\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Note:Predict and call  do tool calling (through text prompting)

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


As is clear in the previous example the LLM first select a relevant tool and then call the tool and give it the inputs and returns the outputs.

Now we want make the RAG functions into the Llama_index callable for the LLM and do the function in the LLM.

In [6]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["D:\\University\\Test\\1.pdf"]).load_data()

In [7]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)

Note: Meta data is some data related to the text like page label and file name and etc. Sometimes analyzing the meta data in the RAG procee can increase the accuracy of the RAG.

In [8]:
print(nodes[6].get_content(metadata_mode="all"))

page_label: 1
file_name: 1.pdf
file_path: D:\University\Test\1.pdf
file_type: application/pdf
file_size: 666613
creation_date: 2024-06-06
last_modified_date: 2024-06-05

The theory of inventive problem solving (TRIZ) is a science that allows creative pro-
blems in any ﬁeld of knowledge to be revealed and solved, while developing creative
(inventive) thinking skills and a creative personality (Altshuller, 2000). Often at the rootof a problem’s solution lies what seems at ﬁrst glance to be a wild idea. TRIZ gives one
the ability not only to be prepared for such ideas, but to create them. TRIZ is the knowl-
edge-based, systematic approach to innovation. TRIZ methods are drawn from analysisof the most innovative inventions in different industries, technologies, and ﬁelds of engin-
eering. These principles can be used to consciously develop a system along its path of tech-
nical evolution. It has been proved that TRIZ is a powerful problem-solving methodologythrough its development over abo

Note:Query engine is a generic interface that allows you to ask question over your data.

A query engine takes in a natural language query, and returns a rich response. It is most often (but not always) built on one or many indexes via retrievers. You can compose multiple query engines to achieve more advanced capability.

In [21]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

Note:

MetadataFilters is  designed to help manage and apply metadata-based filtering to vector stores. 

filters=MetadataFilters.from_dicts: This parameter applies metadata filtering. The MetadataFilters.from_dicts method creates a filter from a list of dictionaries, each specifying a key-value pair for filtering. In this case, the filter will only consider vectors with the metadata key page_label set to "1"

In [22]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=5,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "1"}
        ]
    )
)

response = query_engine.query(
    "WHat is TRIZ and why should we integrate it with the six sigma?", 
)

In [23]:
print(str(response))

TRIZ is an effective method for analyzing customer needs and developing innovative solutions to meet those needs. It provides a structured approach to problem-solving and encourages creative thinking. Integrating TRIZ with Six Sigma can enhance the traditional techniques of Six Sigma by bringing in a systematic innovation methodology that can help identify and resolve conflicts more effectively, leading to improved problem-solving and innovative solutions in various industries, including banking services.


In [24]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_

Note: we can make the meta data filter callable for the LLM and ask the LLM.

In [40]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=5,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)


In [41]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    ''' please consider all pages in this paper and give me a comprehensive definition for the integration of TRIZ and Six Sigma''', 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "integration of TRIZ and Six Sigma", "page_numbers": []}
=== Function Output ===
The integration of TRIZ and Six Sigma has been shown to be effective in improving banking services. This combination enhances traditional Lean Six Sigma techniques by providing innovative solutions to meet customer needs and improve processes.


As is clear here the LLM select the tool and then call and input the tool with the inputs that it has selected for the tool

In [42]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_

We can also add other tools to the tool calling

In [43]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT"
    ),
)

In [45]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    
    ''' please consider all pages in this paper and give me a comprehensive definition for the integration of TRIZ and Six Sigma''', 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "integration of TRIZ and Six Sigma", "page_numbers": []}
=== Function Output ===
The integration of TRIZ and Six Sigma has been shown to be effective in improving banking services. This combination enhances traditional Lean Six Sigma techniques by providing innovative solutions to meet customer needs and improve service quality and speed. The application of Lean Six Sigma methodology with TRIZ has proven to be successful in enhancing the overall performance of banking services.


In [46]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    
    '''give me a comprehensive definition of TRIZ''', 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "TRIZ"}
=== Function Output ===
TRIZ is a problem-solving methodology that allows for creative solutions to be developed by revealing and solving creative problems. It focuses on developing inventive thinking skills and a creative personality. TRIZ is effective in analyzing customer needs and developing innovative solutions to meet those needs. It has been successfully applied in various industries and fields over the past 60 years, providing a systematic approach to innovation. TRIZ helps individuals understand problems as systems, visualize ideal solutions, and enhance product performance by resolving contradictions.


In [47]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_modified_date': '2024-06-05'}
{'page_label': '1', 'file_name': '1.pdf', 'file_path': 'D:\\University\\Test\\1.pdf', 'file_type': 'application/pdf', 'file_size': 666613, 'creation_date': '2024-06-06', 'last_